# Иллюстрируем способ использования предсказания по ряду поправок


### Импортируем модули

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import time
from processing import *

### Загружаем данные заказов и тарифов

In [2]:
snap_shot_file = 'stat_2024_09_12.csv'
pr_df = prepare_prices("тарифы июнь 2024_1.csv")
transfers = pd.read_csv(snap_shot_file,delimiter=';')
transfers['ctime'].min(), transfers['ctime'].max()

('2024-06-23 18:57:16.949313', '2024-09-12 00:15:58.466253')

#### Приводим типы и берём данные за июль-сентябрь и расстояния не ближе 50 км

In [3]:
adjust_types(transfers)
transfers = transfers.sort_values(by=['ctime','delivery_order_id'])
transfers = transfers[(transfers['ctime']>='2024-07-01 00:00:00') & (transfers['distance']>=50)].copy()
wdf, dest = make_working_df(transfers,target_key='price_gross_per_ton',add_month=True,adjust_dist=False, diesel_file=None,diesel_num=1)
wdf.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 726 entries, (Timestamp('2024-07-01 00:00:00'), 47871) to (Timestamp('2024-09-12 00:00:00'), 52016)
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   month        726 non-null    int32  
 1   dist         726 non-null    int64  
 2   target       726 non-null    int64  
 3   targ_price   726 non-null    float64
 4   region_id    726 non-null    int64  
 5   crops_id     726 non-null    int64  
 6   day_of_week  726 non-null    int32  
 7   hours        726 non-null    int64  
 8   dest_id      726 non-null    int64  
dtypes: float64(1), int32(2), int64(6)
memory usage: 72.1 KB


In [4]:
dest

array(['«ЗТК Тамань» ООО', 'АО "ЕЙСКИЙ ПОРТОВЫЙ ЭЛЕВАТОР"',
       'АО "КХП "ТИХОРЕЦКИЙ"', 'АО "ПЛЕМЕННОЙ ЗАВОД "ГАШУНСКИЙ"',
       'АО "РОСТОВСКИЙ ПОРТ" (Четвертый грузовой район)',
       'АО «Ейский морской порт»', 'АО «КСК»', 'АО РАССВЕТ', 'Азов',
       'Архангельский Морской Торговый Порт',
       'Багаевское ХПП Агропром-Импэкс, филиал (Агропромимпэкс, Элеватор)',
       'Гигант', 'ЗЕРНОВОЙ ТЕРМИНАЛ "СТЕПЬ"', 'Зерновой терминал «СТЕПЬ»',
       'Новатор', 'ООО  ПКФ «Порт Корвет»', 'ООО "ВИРТУМ" (Азов)',
       'ООО "ВНЕШТОРГПОРТ"', 'ООО "КАНЕВСКОЙ ЭЛЕВАТОР"',
       'ООО "МАСЛОЗАВОД КРОНОС"',
       'ООО "МОРСКОЙ ГРУЗОВОЙ ТЕРМИНАЛ ИМПЕРИАЛ"',
       'ООО "МЭЗ" (филиал Гигант)', 'ООО "РОСТОВСКИЙ ЗЕРНОВОЙ ТЕРМИНАЛ"',
       'ООО "ЮГ РУСИ"', 'ООО "ЮЖНЫЙ МОРСКОЙ ПОРТ"', 'ООО «НЗТ»',
       'ООО Благодарное', 'ООО ИПАТОВСКИЙ КОМБИНАТ ХЛЕБОПРОДУКТОВ',
       'ООО ЛАБИНСКИЙ ЭЛЕВАТОР', 'ООО МЭЗ Ресурс ОП Усть-Лабинский МЭЗ',
       'ООО ПТ ПРИБОЙ', 'ООО ЧЕРНЫШКОВСКИЙ ЭЛЕВАТОР', 'ПАО «Н

Очищаем набор данных от выбросов и от перевозок из регионов, для которых отсутсвуют тарифы

In [5]:
wdf = refine_df(wdf,pr_df)

#### Считаем количество данных по культурам и регионам

In [6]:
counts = wdf.groupby(['region_id','crops_id']).size().reset_index().rename(columns={0: 'count'})
counts

,region_id,crops_id,count
0,8,1,2
1,8,2,1
2,23,1,13
3,23,2,371
4,23,3,13
5,23,6,71
6,23,8,1
7,23,16,1
8,23,24,2
9,26,1,15


In [7]:
# посмотрим, для каких сочетаний регион-культура имеет смысл применять времнные ряды из наличия хотя бы пары перевозок на 3 дня
# грубо 74 дня, делим на 3 и умножаем на 2
# соотношение "на грани", лучше - больше 
# в остальных случаях будем использовать только прецеденты
sig_counts = counts[counts['count']>50]
sig_counts

,region_id,crops_id,count
3,23,2,371
5,23,6,71
10,26,2,74
28,61,2,78


In [8]:
reg_counts = counts.groupby(['region_id']).sum().loc[:,['count']]
reg_counts

,count
region_id,
8,3
23,472
26,105
29,5
30,1
34,4
36,4
61,98


In [9]:
sig_regs = list(reg_counts[reg_counts['count']>10].index.get_level_values(0))
sig_regs

[23, 26, 61]

In [10]:
wdf.tail(20)

month  dist  target   targ_price  region_id  crops_id  \
date       id                                                             
2024-09-11 51859      9   345  130000   376.811594         23         2   
           51887      9   423  150000   354.609929         23         2   
           51907      9   390  164000   420.512821         23         2   
           51911      9   595  128000   215.126050         29         1   
           51924      9    60   94000  1566.666667         23         3   
2024-09-12 51799      9   260  146000   561.538462         23         2   
           51818      9   235  120000   510.638298         23         2   
           51864      9   362  140000   386.740331         23         2   
           51865      9   252  125000   496.031746         23         2   
           51869      9   323  125000   386.996904         23         2   
           51904      9   371  160000   431.266846         23         2   
           51910      9   226  120000   530.973451         23         2   
           51920      9   403  175000   434.243176         26         2   
           51948      9   222  110000   495.495495         23         2   
           51954      9   267  130000   486.891386         23         2   
           51979      9   341  150000   439.882698         23         2   
           51982      9   283  120000   424.028269         23         2   
           51998      9   601  220000   366.056572         26         1   
           52003      9   365  150000   410.958904         23         3   
           52016      9   505  190000   376.237624         61         2   

                  day_of_week  hours  dest_id  
date       id                                  
2024-09-11 51859            2     24        0  
           51887            2     20        0  
           51907            2      3        6  
           51911            2      6       33  
           51924            2      0       32  
2024-09-12 51799            3     68       32  
           51818            3      5       25  
           51864            3     43       25  
           51865            3     38        6  
           51869            3     45        0  
           51904            3      0       25  
           51910            3     26       25  
           51920            3     23        6  
           51948            3     22       25  
           51954            3     20       32  
           51979            3     15       25  
           51982            3      0        0  
           51998            3      0       32  
           52003            3      3       25  
           52016            3      2        0

### Задаём наборы параметров фильтрации и моделей

In [11]:
fp = filter_par_df(np.arange(1,4),np.append(np.arange(2,11),0),np.append(np.arange(2,6),0))
fp

,use_filter,window,filter_window,filter_order
0,True,1,3,2
1,True,1,4,2
2,True,1,4,3
3,True,1,5,2
4,True,1,5,3
...,...,...,...,...
76,True,3,10,4
77,True,3,10,5
78,False,1,0,0
79,False,2,0,0


In [12]:
mp = model_par_df(['fourier','poly'],np.arange(2,20),np.arange(0,10),np.append(np.arange(4,9),0),np.arange(0,7))
mp

,variant,train_window,n_harm,period,poly_order
0,fourier,2,1,0,0
1,fourier,3,1,0,0
2,fourier,3,2,0,0
3,fourier,4,1,0,0
4,fourier,4,2,0,0
...,...,...,...,...,...
145,poly,8,0,0,5
146,poly,8,0,0,6
147,poly,9,0,0,5
148,poly,9,0,0,6


In [13]:
filt_pars_var3, mod_pars_var3 = get_pars(fp,mp,52,127)
prefix3 = 'var3'
form_type3 = 'k1'
filt_pars_var2, mod_pars_var2 = get_pars(fp,mp,67,23)
prefix2 = 'var2'
form_type2 = 'k0k1'
all_pars = [(filt_pars_var3, mod_pars_var3,form_type3,prefix3),(filt_pars_var2, mod_pars_var2,form_type2,prefix2)]

### Задаём функцию предсказания с использованием временных рядов по принципу скользящего окна

In [14]:
#предсказание идёт на одну дату с заданным смещением (горизонтом прогнозирования) относительно конца обучающей выборки
#затем окно выборки и дата предсказания смещаются

#pred_start - дата начала предсказаний,
#t_df - набор данных с тарифами,
#region_id,crops_id - выбранные регион и культура,
#horizon - горизонт прогнозирования,
#filt_pars - параметры фильтрации,
#model_pars - параметры модели прогноза,
#form_type - тип формулы цены,
#per_tonn_km - признак расчёта удельной цены
#prefix - префикс названия столбца результата
#min_samples - минимально требуемое число перевозок для подбора коэффициентов
#min_dists - минимально требуемое число перевозок на разные дистанции для подбора коэффициентов
def slide_predict(df,*,form_type,min_samples,min_dists,pred_start,t_df,region_id,crops_id,horizon,\
                  filt_pars, model_pars,per_tonn_km = 'True',prefix = ''):
    #создаём экземпляр модели
    predictor = SeriesPredictor(tarif_df = pr_df,fpars = filt_pars, mpars = model_pars, region_id = region_id, form_type=form_type)
    #делаем выборку по региону и культуре
    rdf = df_region_crop(df,region_id = region_id, crops_id = crops_id)
    #распаковываем параметры
    _,sample_window,_,_ = predictor.fpars
    _, train_window, _, _, _ = predictor.mpars
    #считаем даты и дельты
    #смещение конца обучающей выборки относительно начала
    train_delta = pd.Timedelta(days = sample_window+train_window-2)
    #смещение даты предсказания относительно конца обучающей выборки
    test_delta = pd.Timedelta(days = horizon)
    #дата начала самой первой обучающей выборки
    start_date =  pd.to_datetime(pred_start) - train_delta - test_delta
    #дата последнего предсказания
    pred_end = rdf.index.get_level_values(0).max()
    #дата начала последней обучающей выборки
    end_date = pred_end - train_delta - test_delta
    #диапазон дат начала обучающих выборок
    date_range = pd.date_range(start_date,end_date)
    #диапазон дат предсказания
    pred_range = pd.date_range(pred_start,pred_end)
    #наименование столбца результата
    res_key = pred_key(prefix,horizon)
    rdf[res_key] = None
    #цикл сдвига окна
    for date in date_range:
        #формируем тестовый набор на горизонт прогнозирования
        test_date = date + train_delta + test_delta
        test_df = rdf[rdf.index.get_level_values(0)==test_date].copy()
        #print(test_date,len(test_df))
        #если есть, что предсказывать
        if len(test_df)>0:            
            #формируем обучающий набор данных
            train_end = date + train_delta;
            #print(date,train_end)
            train_df = rdf[(rdf.index.get_level_values(0)>=date)&(rdf.index.get_level_values(0)<=train_end)].copy()
            train_df.reindex(pd.date_range(date,train_end),level = 0)
            #print(train_df)
            try:
                predictor.fit(train_df,date = train_end)
                #рассчитываем результат на дату прогноза
                res = predictor.predict(horizon,horizon, df = test_df)
            except:
                print('не хватает данных для прогноза на дату',test_date)
            else:
                #при необходимости, приводим к коп/(т*км)
                if per_tonn_km:
                    res['target_pred'] = res['target_pred']/res['dist']
                #заносим в соответствующий столбец
                rdf.loc[res.index,[res_key]] = res['target_pred']
    if per_tonn_km:
        rdf['err_'+res_key] = rdf[res_key]-rdf['targ_price']
    else:
        rdf['err_'+res_key] = rdf[res_key]-rdf['target']
    return rdf

### Задаём функцию обработки данных с использованием рекомендуемых вариантов предсказания на основе временных рядов

In [15]:
#процедура тестирования моделей на основе временных рядов с заданным горизонтом прогнозирования по заданному списку сочетаний регион-культура
#pred_start - дата начала предсказаний,
#t_df - набор данных с тарифами,
#combinations - сочетания региона и культуры
#horizon - горизонт прогнозирования,
#pars - список кортежей с параметрами фильтрации, модели прогноза, типом формулы цены, префиксом столбца результата
#per_tonn_km - признак расчёта удельной цены
#min_samples - минимально требуемое число перевозок для подбора коэффициентов
#min_dists - минимально требуемое число перевозок на разные дистанции для подбора коэффициентов

def evaluate_time_series(df,*,t_df,pred_start,combinations,horizon,pars,per_tonn_km,min_samples,min_dists):
    res = df.copy()
    print('Prediction for horizon = ',horizon,'days')
    print('')
    for par in pars:
        filt_pars, mod_pars,form_type,prefix = par
        res_key = pred_key(prefix,horizon)
        err_key = 'err_'+res_key
        print('Prediction by ', prefix,', ','formula_type is ',form_type)
        res[res_key] = None
        res[err_key] = None
        for index, row in combinations.iterrows():
            region_id = row['region_id']
            crops_id = row['crops_id']
            print('region = ',region_id,',','crops = ',crops_id)
            tmp = slide_predict(df,form_type = form_type, min_samples = min_samples,min_dists = min_dists, pred_start = pred_start,t_df = t_df,region_id = region_id,crops_id = crops_id,horizon = horizon,\
                  filt_pars = filt_pars, model_pars = mod_pars ,per_tonn_km = per_tonn_km,prefix = prefix)
            res.loc[tmp.dropna().index,[res_key]] = tmp.dropna()[res_key]
            res.loc[tmp.dropna().index,[err_key]] = tmp.dropna()[err_key]
    return res


### Задаём функцию предсказания с использованием прецедентов по принципу расширяющегося окна

In [16]:
#предсказание идёт на одну дату с заданным смещением (горизонтом прогнозирования) относительно конца обучающей выборки
#затем окно выборки расширяется, а дата предсказания смещается

#pred_start - дата начала предсказаний,
#t_df - набор данных с тарифами,
#region_id - выбранный регион,
#horizon - горизонт прогнозирования,
#per_tonn_km - признак расчёта удельной цены
#prefix - префикс названия столбца результата
def roll_predict(df,*,pred_start,t_df,region_id = -1,horizon, per_tonn_km = 'True',prefix = 'precedent'):
    rdf = df.copy()
    #создаём экземпляр модели
    predictor = PrecedentPredictor(tarif_df = pr_df, eps = 0.1, region_id = region_id)
    #считаем даты и дельты
    #смещение даты предсказания относительно конца обучающей выборки
    test_delta = pd.Timedelta(days = horizon)
    #дата конца самой первой обучающей выборки
    start_date =  pd.to_datetime(pred_start) - test_delta
    #дата начала самой первой обучающей выборки
    train_start =  rdf.index.get_level_values(0).min()
    #дата последнего предсказания
    pred_end = rdf.index.get_level_values(0).max()
    #дата конца последней обучающей выборки
    end_date = pred_end - test_delta
    #диапазон дат конца обучающих выборок
    date_range = pd.date_range(start_date,end_date)
    #диапазон дат предсказания
    pred_range = pd.date_range(pred_start,pred_end)
    #наименование столбца результата
    res_key = pred_key(prefix,horizon)
    err_key = 'err_'+ res_key    
    #цикл расширения окна
    for date in date_range:
        #формируем тестовый набор на горизонт прогнозирования
        test_date = date + test_delta
        test_df = rdf[rdf.index.get_level_values(0)==test_date].loc[:,['dist','target','region_id','crops_id','dest_id','targ_price']].copy()
        test_df['date']=test_df.index.get_level_values(0)
        #print(test_date,len(test_df))
        #если есть, что предсказывать
        if len(test_df)>0:            
            #формируем обучающий набор данных
            #print(train_start, date)
            train_df = rdf[(rdf.index.get_level_values(0)>=train_start)&(rdf.index.get_level_values(0)<=date)].copy()
            #print(train_df)
            predictor.fit(train_df)
            #рассчитываем результат на дату прогноза
            test_df[res_key] = predictor.predict(test_df)
            #при необходимости, приводим к коп/(т*км)
            if per_tonn_km:
                test_df[res_key] = test_df[res_key]/test_df['dist']
                test_df[err_key] = test_df[res_key]-test_df['targ_price']
            else:
                test_df[err_key] = test_df[res_key]-test_df['target']
            #заносим в соответствующий столбец
            rdf.loc[test_df.dropna().index,[res_key]] = test_df.dropna()[res_key]
            rdf.loc[test_df.dropna().index,[err_key]] = test_df.dropna()[err_key]
    return rdf

### Задаём функцию обработки данных с использованием прецедентов

In [17]:
#процедура тестирования моделей на основе прецедентов с заданным горизонтом прогнозирования
#pred_start - дата начала предсказаний,
#t_df - набор данных с тарифами,
#regions - выбранные регионы для работы на подвыборках,
#horizon - горизонт прогнозирования,
#prefix - префикс названия столбца результата
#per_tonn_km - признак расчёта удельной цены

def evaluate_precedents(df,*,t_df,pred_start,regions,horizon,prefix = 'precedent',per_tonn_km):
    print('Prediction for horizon = ',horizon,'days')
    print('')
    res = df.copy()
    print('Prediction for minor regions')
    res_key = pred_key(prefix,horizon)
    err_key = 'err_'+res_key
    res[res_key] = None
    res[err_key] = None
    res = roll_predict(res,pred_start = pred_start,t_df = t_df,horizon = horizon,per_tonn_km = per_tonn_km,prefix = prefix)
    for region_id in regions:
        print('Prediction for region ', region_id)
        res = roll_predict(res,pred_start = pred_start,t_df = t_df,region_id = region_id,horizon = horizon,per_tonn_km = per_tonn_km,prefix = prefix)
    return res

### Формируем прогноз с горизонтом 3 дня

In [18]:
%%capture --no-stdout --no-display
pred3 = evaluate_time_series(wdf, t_df = pr_df,pred_start = '2024-08-01', combinations = sig_counts, horizon = 3, pars = all_pars, per_tonn_km = True, min_samples = 2, min_dists = 2)

Prediction for horizon =  3 days

Prediction by  var3 ,  formula_type is  k1
region =  23 , crops =  2
region =  23 , crops =  6
не хватает данных для прогноза на дату 2024-08-10 00:00:00
не хватает данных для прогноза на дату 2024-08-28 00:00:00
region =  26 , crops =  2
не хватает данных для прогноза на дату 2024-08-01 00:00:00
не хватает данных для прогноза на дату 2024-08-02 00:00:00
не хватает данных для прогноза на дату 2024-08-03 00:00:00
не хватает данных для прогноза на дату 2024-08-04 00:00:00
не хватает данных для прогноза на дату 2024-08-05 00:00:00
не хватает данных для прогноза на дату 2024-08-07 00:00:00
не хватает данных для прогноза на дату 2024-08-23 00:00:00
не хватает данных для прогноза на дату 2024-08-25 00:00:00
не хватает данных для прогноза на дату 2024-09-12 00:00:00
region =  61 , crops =  2
не хватает данных для прогноза на дату 2024-08-05 00:00:00
не хватает данных для прогноза на дату 2024-08-08 00:00:00
не хватает данных для прогноза на дату 2024-08-09 00

In [19]:
pred3 =  evaluate_precedents(pred3,pred_start = '2024-08-01',t_df = pr_df,regions = sig_regs,horizon =3,per_tonn_km = True)

Prediction for horizon =  3 days

Prediction for minor regions
Prediction for region  23
Prediction for region  26
Prediction for region  61


In [20]:
pred3.tail(20)

month  dist  target   targ_price  region_id  crops_id  \
date       id                                                             
2024-09-11 51859      9   345  130000   376.811594         23         2   
           51887      9   423  150000   354.609929         23         2   
           51907      9   390  164000   420.512821         23         2   
           51911      9   595  128000   215.126050         29         1   
           51924      9    60   94000  1566.666667         23         3   
2024-09-12 51799      9   260  146000   561.538462         23         2   
           51818      9   235  120000   510.638298         23         2   
           51864      9   362  140000   386.740331         23         2   
           51865      9   252  125000   496.031746         23         2   
           51869      9   323  125000   386.996904         23         2   
           51904      9   371  160000   431.266846         23         2   
           51910      9   226  120000   530.973451         23         2   
           51920      9   403  175000   434.243176         26         2   
           51948      9   222  110000   495.495495         23         2   
           51954      9   267  130000   486.891386         23         2   
           51979      9   341  150000   439.882698         23         2   
           51982      9   283  120000   424.028269         23         2   
           51998      9   601  220000   366.056572         26         1   
           52003      9   365  150000   410.958904         23         3   
           52016      9   505  190000   376.237624         61         2   

                  day_of_week  hours  dest_id  var3_pred3 err_var3_pred3  \
date       id                                                              
2024-09-11 51859            2     24        0  417.359591      40.547996   
           51887            2     20        0  394.752613      40.142684   
           51907            2      3        6  398.230609     -22.282211   
           51911            2      6       33        None           None   
           51924            2      0       32        None           None   
2024-09-12 51799            3     68       32  456.982084    -104.556378   
           51818            3      5       25  480.484019     -30.154279   
           51864            3     43       25  407.366886      20.626554   
           51865            3     38        6  456.982084     -39.049662   
           51869            3     45        0  427.387053      40.390149   
           51904            3      0       25  407.366886      -23.89996   
           51910            3     26       25  480.484019     -50.489432   
           51920            3     23        6        None           None   
           51948            3     22       25  516.172144      20.676648   
           51954            3     20       32  456.982084     -29.909302   
           51979            3     15       25  417.812191     -22.070507   
           51982            3      0        0  449.148105      25.119836   
           51998            3      0       32        None           None   
           52003            3      3       25        None           None   
           52016            3      2        0  379.811028       3.573404   

                  var2_pred3 err_var2_pred3 precedent_pred3  \
date       id                                                 
2024-09-11 51859   402.70119      25.889596      463.768116   
           51887  392.515995      37.906066      413.711584   
           51907  390.454022     -30.058799      448.717949   
           51911        None           None        8.403361   
           51924        None           None     1566.666667   
2024-09-12 51799  399.547382    -161.991079      553.846154   
           51818  407.606272    -103.032026      476.595745   
           51864  395.529613       8.789281       441.98895   
           51865  392.332593    -103.699154      496.031746   
 

In [21]:
ser3.tail(20)

NameError: name 'ser3' is not defined

In [22]:
pred3.to_csv('3_days_prediction.csv',sep = ';')

### Формируем прогноз с горизонтом 5 дней

In [23]:
%%capture --no-stdout --no-display
pred5 = evaluate_time_series(wdf, t_df = pr_df,pred_start = '2024-08-01', combinations = sig_counts, horizon = 5, pars = all_pars, per_tonn_km = True, min_samples = 2, min_dists = 2)

Prediction for horizon =  5 days

Prediction by  var3 ,  formula_type is  k1
region =  23 , crops =  2
region =  23 , crops =  6
не хватает данных для прогноза на дату 2024-08-02 00:00:00
не хватает данных для прогноза на дату 2024-08-28 00:00:00
region =  26 , crops =  2
не хватает данных для прогноза на дату 2024-08-02 00:00:00
не хватает данных для прогноза на дату 2024-08-03 00:00:00
не хватает данных для прогноза на дату 2024-08-04 00:00:00
не хватает данных для прогноза на дату 2024-08-05 00:00:00
не хватает данных для прогноза на дату 2024-08-06 00:00:00
не хватает данных для прогноза на дату 2024-08-07 00:00:00
не хватает данных для прогноза на дату 2024-08-09 00:00:00
не хватает данных для прогноза на дату 2024-08-24 00:00:00
не хватает данных для прогноза на дату 2024-08-25 00:00:00
не хватает данных для прогноза на дату 2024-08-27 00:00:00
не хватает данных для прогноза на дату 2024-08-31 00:00:00
region =  61 , crops =  2
не хватает данных для прогноза на дату 2024-08-03 00

In [24]:
pred5 =  evaluate_precedents(pred5,pred_start = '2024-08-01',t_df = pr_df,regions = sig_regs,horizon =5,per_tonn_km = True)

Prediction for horizon =  5 days

Prediction for minor regions
Prediction for region  23
Prediction for region  26
Prediction for region  61


In [25]:
pred5.tail(20)

month  dist  target   targ_price  region_id  crops_id  \
date       id                                                             
2024-09-11 51859      9   345  130000   376.811594         23         2   
           51887      9   423  150000   354.609929         23         2   
           51907      9   390  164000   420.512821         23         2   
           51911      9   595  128000   215.126050         29         1   
           51924      9    60   94000  1566.666667         23         3   
2024-09-12 51799      9   260  146000   561.538462         23         2   
           51818      9   235  120000   510.638298         23         2   
           51864      9   362  140000   386.740331         23         2   
           51865      9   252  125000   496.031746         23         2   
           51869      9   323  125000   386.996904         23         2   
           51904      9   371  160000   431.266846         23         2   
           51910      9   226  120000   530.973451         23         2   
           51920      9   403  175000   434.243176         26         2   
           51948      9   222  110000   495.495495         23         2   
           51954      9   267  130000   486.891386         23         2   
           51979      9   341  150000   439.882698         23         2   
           51982      9   283  120000   424.028269         23         2   
           51998      9   601  220000   366.056572         26         1   
           52003      9   365  150000   410.958904         23         3   
           52016      9   505  190000   376.237624         61         2   

                  day_of_week  hours  dest_id  var3_pred5 err_var3_pred5  \
date       id                                                              
2024-09-11 51859            2     24        0  424.257852      47.446258   
           51887            2     20        0  401.277218      46.667289   
           51907            2      3        6    404.8127      -15.70012   
           51911            2      6       33        None           None   
           51924            2      0       32        None           None   
2024-09-12 51799            3     68       32  460.866281    -100.672181   
           51818            3      5       25  484.567975     -26.070323   
           51864            3     43       25   410.82937      24.089039   
           51865            3     38        6  460.866281     -35.165465   
           51869            3     45        0  431.019703      44.022799   
           51904            3      0       25   410.82937     -20.437476   
           51910            3     26       25  484.567975     -46.405476   
           51920            3     23        6  440.098874       5.855698   
           51948            3     22       25  520.559437      25.063942   
           51954            3     20       32  460.866281     -26.025105   
           51979            3     15       25  421.363457     -18.519241   
           51982            3      0        0  452.965716      28.937447   
           51998            3      0       32        None           None   
           52003            3      3       25        None           None   
           52016            3      2        0     389.217      12.979376   

                  var2_pred5 err_var2_pred5 precedent_pred5  \
date       id                                                 
2024-09-11 51859  413.361909      36.550314      463.768116   
           51887  387.826415      33.216485      413.711584   
           51907  392.736496     -27.776325      448.717949   
           51911        None           None        8.403361   
           51924        None           None     1566.666667   
2024-09-12 51799  420.867467    -140.670995      553.846154   
           51818  438.407377     -72.230921      476.595745   
           51864  388.109831       1.369499       441.98895   
           51865  418.496265     -77.535481      496.031746   
 

In [26]:
pred5.to_csv('5_days_prediction.csv',sep = ';')

### Формируем прогноз с горизонтом 7 дней

In [27]:
%%capture --no-stdout --no-display
pred7 = evaluate_time_series(wdf, t_df = pr_df,pred_start = '2024-08-01', combinations = sig_counts, horizon = 7, pars = all_pars, per_tonn_km = True, min_samples = 2, min_dists = 2)

Prediction for horizon =  7 days

Prediction by  var3 ,  formula_type is  k1
region =  23 , crops =  2
region =  23 , crops =  6
не хватает данных для прогноза на дату 2024-08-02 00:00:00
не хватает данных для прогноза на дату 2024-08-04 00:00:00
не хватает данных для прогноза на дату 2024-08-28 00:00:00
region =  26 , crops =  2
не хватает данных для прогноза на дату 2024-08-01 00:00:00
не хватает данных для прогноза на дату 2024-08-04 00:00:00
не хватает данных для прогноза на дату 2024-08-05 00:00:00
не хватает данных для прогноза на дату 2024-08-06 00:00:00
не хватает данных для прогноза на дату 2024-08-07 00:00:00
не хватает данных для прогноза на дату 2024-08-08 00:00:00
не хватает данных для прогноза на дату 2024-08-09 00:00:00
не хватает данных для прогноза на дату 2024-08-11 00:00:00
не хватает данных для прогноза на дату 2024-08-26 00:00:00
не хватает данных для прогноза на дату 2024-08-27 00:00:00
не хватает данных для прогноза на дату 2024-09-02 00:00:00
не хватает данных д

In [28]:
pred7 =  evaluate_precedents(pred7,pred_start = '2024-08-01',t_df = pr_df,regions = sig_regs,horizon = 7,per_tonn_km = True)

Prediction for horizon =  7 days

Prediction for minor regions
Prediction for region  23
Prediction for region  26
Prediction for region  61


In [29]:
pred7.tail(20)

month  dist  target   targ_price  region_id  crops_id  \
date       id                                                             
2024-09-11 51859      9   345  130000   376.811594         23         2   
           51887      9   423  150000   354.609929         23         2   
           51907      9   390  164000   420.512821         23         2   
           51911      9   595  128000   215.126050         29         1   
           51924      9    60   94000  1566.666667         23         3   
2024-09-12 51799      9   260  146000   561.538462         23         2   
           51818      9   235  120000   510.638298         23         2   
           51864      9   362  140000   386.740331         23         2   
           51865      9   252  125000   496.031746         23         2   
           51869      9   323  125000   386.996904         23         2   
           51904      9   371  160000   431.266846         23         2   
           51910      9   226  120000   530.973451         23         2   
           51920      9   403  175000   434.243176         26         2   
           51948      9   222  110000   495.495495         23         2   
           51954      9   267  130000   486.891386         23         2   
           51979      9   341  150000   439.882698         23         2   
           51982      9   283  120000   424.028269         23         2   
           51998      9   601  220000   366.056572         26         1   
           52003      9   365  150000   410.958904         23         3   
           52016      9   505  190000   376.237624         61         2   

                  day_of_week  hours  dest_id  var3_pred7 err_var3_pred7  \
date       id                                                              
2024-09-11 51859            2     24        0  440.035837      63.224243   
           51887            2     20        0  416.200562      61.590633   
           51907            2      3        6  419.867528      -0.645293   
           51911            2      6       33        None           None   
           51924            2      0       32        None           None   
2024-09-12 51799            3     68       32  473.253265     -88.285197   
           51818            3      5       25  497.592004     -13.046294   
           51864            3     43       25  421.871482       35.13115   
           51865            3     38        6  473.253265     -22.778481   
           51869            3     45        0  442.604482      55.607578   
           51904            3      0       25  421.871482      -9.395365   
           51910            3     26       25  497.592004     -33.381447   
           51920            3     23        6  428.998203      -5.244973   
           51948            3     22       25  534.550831      39.055335   
           51954            3     20       32  473.253265     -13.638121   
           51979            3     15       25  432.688699      -7.193999   
           51982            3      0        0  465.140352      41.112083   
           51998            3      0       32        None           None   
           52003            3      3       25        None           None   
           52016            3      2        0  406.301585      30.063962   

                  var2_pred7 err_var2_pred7 precedent_pred7  \
date       id                                                 
2024-09-11 51859  449.815588      73.003993      463.768116   
           51887  412.563429        57.9535      413.711584   
           51907  422.316492       1.803671      448.717949   
           51911        None           None        8.403361   
           51924        None           None     1566.666667   
2024-09-12 51799  480.081845     -81.456617      553.846154   
           51818  506.772106      -3.866192      476.595745   
           51864  421.654328      34.913996       441.98895   
           51865  481.237399     -14.794347      496.031746   
 

In [30]:
pred7.to_csv('7_days_prediction.csv',sep = ';')